<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-class-structure" data-toc-modified-id="Data-class-structure-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data class structure</a></span></li><li><span><a href="#How-to-create-a-Data-object" data-toc-modified-id="How-to-create-a-Data-object-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>How to create a Data object</a></span><ul class="toc-item"><li><span><a href="#How-to-create-a-Data1D-object" data-toc-modified-id="How-to-create-a-Data1D-object-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>How to create a Data1D object</a></span></li><li><span><a href="#How-to-create-a-DataLinspace-object" data-toc-modified-id="How-to-create-a-DataLinspace-object-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>How to create a DataLinspace object</a></span></li><li><span><a href="#How-to-create-a-DataTime-object" data-toc-modified-id="How-to-create-a-DataTime-object-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>How to create a DataTime object</a></span></li><li><span><a href="#How-to-create-a-DataFreq-object" data-toc-modified-id="How-to-create-a-DataFreq-object-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>How to create a DataFreq object</a></span></li></ul></li></ul></div>

# How to create Data objects

The SciDataTool python module has been created to ease the handling of scientific data, and considerately simplify plot commands. It unifies the extraction of relevant data (e.g. slices), whether they are stored in the time/space or in the frequency domain. The call to Fourier Transform functions is transparent, although it still can be parameterized through the use of a dictionary.

This tutorial explains the structure of the `Data` classes, then shows how to create axes and fields objects.

The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/SciDataTool/tree/master/Tutorials/tuto_Create.ipynb).

The following example demonstrates the syntax to quickly create a 2D data field depending on time and angle. For more details, see below.

In [1]:
# Add SciDataTool to the Python path
import sys
sys.path.append('../..')
from numpy import pi
from pandas import read_excel

from SciDataTool.Classes.Data1D import Data1D
from SciDataTool.Classes.DataTime import DataTime
from SciDataTool.Tests import DATA_DIR

xls_file = join(DATA_DIR, "tutorials_data.xlsx")
time = read_excel(xls_file, "time").to_numpy()

print(time)


ModuleNotFoundError: No module named 'pyleecan'

## Data class structure
The `Data` class is composed of:
- classes describing axes (`Data1D` and `DataLinspace`)
- classes describing fields stored in the time/space domain (`DataTime`) or in the frequential domain (`DataFreq`).

The following UML summarizes this structure:

<div>
<img src="_static/UML_Data_Object.png" width="450"/>
</div>

## How to create a Data object
At the end of a simulation, the idea is to store the resulting fields into `Data` objects before post-processing them. To do so, the axes have to be created first.

### How to create a Data1D object
A `Data1D` object has four properties: a `name`, a `unit`, a dictionary for `symmetries`, and a numpy array for the `values`.

- The `name` is a string and can be user-defined, although restrictions appear when one wants to perform Fourier transforms. In this case, the correspondance with the frequency domain names must exist among the predefined correspondances:
  + `"time"` &harr; `"freqs"`
  + `"angle"` &harr; `"wavenumber"`
  
$\color{red}{\text{To be expanded + possibility to manually enter relation?}}$

- The `unit` is also a string, and corresponds to the symbol of the unit (*dimless, m, rad, °, g, s, min, h, Hz, rpm, degC, A, J, W, N, C, T, G, V, F, H, Ohm, At, Wb, Mx*), with a prefix (*k*, *h*, *da*, *d*, *c*, *m*, etc.). Composed units are also available (e.g. `"mm/s^2"`). It is used for labelling plot axes, so that it is best to use a LaTeX formatting.


- The `symmetries` are used to reduce storage: a signal can be periodic ($f(t+T)=f(t)$) or antiperiodic ($f(t+T)=-f(t)$). The symmetries dictionary must be filled with the name of the axis and a dictionary of its symmetry (`{"period": n}` or `{"antiperiod": n}`, with *n* the number of periods in the complete signal. Note that the symmetries dictionary must be shared with the field itself (`DataTime` or `DataFreq`).


In the following example, the time vector is imported from an xls file and stored in a `Data1D` object. A second object is created using a 1/3 periodicity to demonstrate the use of the symmetries dictionary:

In [1]:
# Add pyleecan to the Python path
import sys
sys.path.append('../..')

from os.path import join
from numpy import squeeze

from pyleecan.Classes.Data1D import Data1D
from pyleecan.Classes.ImportMatrixXls import ImportMatrixXls
from pyleecan.Tests import DATA_DIR

xls_file_time = join(DATA_DIR, "default_proj_time.xlsx")
time = squeeze(ImportMatrixXls(file_path=xls_file_time, sheet="default_proj_time").get_data())
time_reduced = time[0:672]

Time = Data1D(name="time", unit="s", symmetries={}, values=time)

Time_reduced = Data1D(name="time", unit="s", symmetries={"time": {"period": 3}}, values=time_reduced)

### How to create a DataLinspace object
Axes often have a regular distribution, so that the use of `DataLinspace` allows to reduce the storage.

A `DataLinspace` object has five properties instead of the `values` array: `initial`, `final`, `step` and `number` allow to define the linspace vector (3 out of these 4 suffice), and `include_endpoint` is a boolean used to indicate whether the final point should be included or not.

In the following example, an angle vector is defined as a linspace:


In [2]:
from numpy import pi
from pyleecan.Classes.DataLinspace import DataLinspace

Angle = DataLinspace(
            name="angle",
            unit="rad",
            symmetries={},
            initial=0,
            final=2*pi,
            number=2016,
            include_endpoint=False,
        )

### How to create a DataTime object
Once the axes have been created, the objects for the fields can be created. If the field is defined in the time/space domain, the `DataTime` class has to be used.

5 additional properties must be defined (note that `name` will be used in the titles of the plots, and `symmetries` should correspond to the axes symmetries):
- `symbol` is the LaTeX formatted string which will be used as a label in the plots
- `axes` is a list of `Data1D` or `DataLinspace` objects corresponding to the axes used in the field
- `normalizations` is a dictionary prescribing the values to be used to normalize the field (through the use of a `is_norm` flag which we will see later) or one of its axes: use `{"unit": X0}` for a field normalization (e.g. `normalizations={"T": 0.8}`), or the name of the normalized axis (e.g. `normalizations={"elec_order": 60}`)
- `FT_parameters` is a dictionary of the type of Fourier Transform to use (`type`, `window`, `window_width`, etc.)
- `values` is a numpy array

See below two examples of `DataTime`creation, without and with symmetry:

In [3]:
from pyleecan.Classes.DataTime import DataTime

xls_file_Br = join(DATA_DIR, "default_proj_Br_time_angle.xlsx")
field = ImportMatrixXls(file_path=xls_file_Br, sheet="default_proj_Br_time_angle").get_data()
field_reduced = field[0:672, :]

Br = DataTime(
            name="Airgap radial flux density",
            unit="T",
            symmetries={},
            symbol="B_r",
            axes=[Time, Angle],
            normalizations={"elec_order": 60, "space_order": 3},
            values=field,
        )

Br_reduced = DataTime(
            name="Airgap radial flux density",
            unit="T",
            symmetries={"time": {"period": 3}},
            symbol="B_r",
            axes=[Time, Angle],
            normalizations={"elec_order": 60, "space_order": 3},
            values=field_reduced,
        )

### How to create a DataFreq object
If one prefers to store data in the frequency domain, for example because most postprocessings will handle spectra, the `DataFreq` class can be used.

The definition is similar to the `DataTime` one: the axes now have to be frequencies or wavenumbers (see following example, with an import from Matlab files).

In [4]:
from pyleecan.Classes.DataFreq import DataFreq
from pyleecan.Classes.ImportMatlab import ImportMatlab

mat_file_Br_cfft2 = join(DATA_DIR, "default_proj_Br_cfft2.mat")
mat_file_Brfreqs = join(DATA_DIR, "default_proj_Brfreqs.mat")
mat_file_Brwavenumber = join(DATA_DIR, "default_proj_Brwavenumber.mat")

Br_cfft2 = squeeze(ImportMatlab(file_path=mat_file_Br_cfft2, var_name="Fwr").get_data())
freqs_Br = squeeze(ImportMatlab(file_path=mat_file_Brfreqs, var_name="freqs").get_data())
wavenumber = squeeze(ImportMatlab(file_path=mat_file_Brwavenumber, var_name="orders").get_data())

Freqs = Data1D(name="freqs", unit="Hz", symmetries={}, values=freqs_Br,)

Wavenumber = Data1D(name="wavenumber", unit="", symmetries={}, values=wavenumber)

Br_fft = DataFreq(
            name="Airgap radial flux density",
            unit="T",
            symmetries={},
            symbol="B_r",
            axes=[Freqs, Wavenumber],
            normalizations={},
            values=Br_cfft2,
        )